
### *Student Name: Krishnakanth Naik Jarapala*
### *NU-ID: 002724795*



# Direct Preference Optimiation - Dataset Generation

**Logging into huggingFace to get access to dataset and models**

In [ ]:
!git config --global credential.helper store

# Install the huggingface_hub library
!pip install huggingface_hub -q

In [ ]:
# Import the notebook_login method
from huggingface_hub import notebook_login

# Log in interactively
notebook_login()

#### 1: Load the Lima datasets instruction

In [ ]:
!pip install datasets -q --no-cache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 178.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 234.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 204.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 276.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 330.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 217.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x

In [ ]:
from datasets import load_dataset
# Load Lima Dataset
ds = load_dataset("GAIR/lima")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

**Sample 50 instructions**

In [ ]:
import random

random.seed(42)

# Load conversations from the train dataset
conversations = ds['train']['conversations']

# Sample 50 random instructions
sampled_convs = random.sample(conversations, 50)

sample_conv_list = []

# Display the sampled conversations
for i, conversation in enumerate(sampled_convs, 1):
    sample_conv_list.append(conversation[0])
    print(f"{i} --> {conversation[0]}\n")

1 --> Can I spend the night alone in a tent in a forest outside Stockholm in -20°C without risking my life?

The backstory
From the end of January, I'm starting my studies in a suburb of Stockholm. I've decided to, if it turns out plausible, not rent an apartment, but live in a tent. (This is not out of frugality, but out of a will to try something new.)
I do have friends who I could visit once a week or so to prepare food and wash my clothes, so I think I can solve the practical problems, or at least those that I've come to think of. I'd camp in one of the forests, maybe 1 km from "civilisation". I'd have access to showers etc at university every day.
However: I don't want to freeze to death in my sleep! That's very important to me. I've read that the nights can get as cold as -20°C (-4°F). With the proper preparations, would this be a plausible way of living, at least for a month or so?
I do have camping experience, and have been hiking for three weeks, but only in summer.

2 --> Thi

In [ ]:
import json

# output filename
output_path = '50_sampled_conversations.json'

# Save the sampled conversations to a JSON file
with open(output_path, mode='w', encoding='utf-8') as file:
    json.dump(sample_conv_list, file)

#### 2 - Load Mistral-7B-Instruct-v0.2

* Loading the 4-bit Quantized Model to reduce its memory footprint and computational requirements, making it easier to load onto a GPU with limited memory.

In [ ]:
!pip install git+https://github.com/huggingface/transformers -q peft  accelerate bitsandbytes safetensors sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 16.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

    return model

def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1
    return tokenizer


model = load_quantized_model(model_name)

tokenizer = initialize_tokenizer(model_name)

# Define stop token ids
stop_token_ids = [0]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

**Testing response generation**

In [ ]:
def generate_response(prompt):
  text = f"[INST] {prompt} [/INST]"
  encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
  model_input = encoded.to(model.device)
  generated_ids = model.generate(**model_input, max_new_tokens=1000, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0].replace(text, '').strip()

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
prompt = "What is Machine Learning?"
response = generate_response(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Machine Learning ( Question: what-is-machine-learning) is a subset of artificial intelligence (AI) that provides systems the ability to learn and improve from experience without being explicitly programmed. It focuses on the development of computer programs that can access data and use it to learn for themselves.

The process of learning begins with training, using a set of data, called a training set. The model learns the invariant features from this data that make it identifiable from other inputs. Once this is completed, the model can correctly identify or make decisions based on new input data, known as test data. The primary aim is to allow the computers to learn automatically without human intervention or assistance and adjust actions accordingly.

Machine learning models can be classified into several types, including supervised learning, unsupervised learning, semi-supervised learning, and reinforcement learning. Each approach has its strengths and challenges, and their applica

### 3 - Generate 5 Responses for each Instruction using mistralai/Mistral-7B-Instruct-v0.2

In [ ]:
import json

results = {}

counter = 0
for i, conversation in enumerate(sampled_convs, 1):
    responses = []
    for _ in range(5):
        counter += 1
        response = generate_response(conversation[0])
        print(counter)
        responses.append(response)
    results[i] = responses

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

247


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


248


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


249
250


In [ ]:
# Save results in a JSON file
with open('generated_responses.json', 'w') as f:
    json.dump(results, f, indent=4)

## 4 - Generating the Preference Dataset

We start by referring to both the instructions and the generated set of 5 responses for each given instruction. Next, we utilize the `llm-blender/PairRM` model to rank these responses, which allows us to create the preference dataset.

### Laod and configure PairRM

In [ ]:
!pip install llm-blender -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00


In [ ]:
import llm_blender
blender = llm_blender.Blender()
blender.loadranker("llm-blender/PairRM") # load PairRM

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

ranker_config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/13.7k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Successfully loaded ranker from  /root/.cache/huggingface/hub/llm-blender/PairRM


In [ ]:
inputs = ["What time is the meeting?", "Can you recommend a good book?"]
candidates_texts = [["It's at 10 AM.", "The meeting is at 2 PM.", "I'll check and let you know."],
                    ["Try 'The Great Gatsby'.", "How about '1984'?", "I’m not sure, maybe check online reviews."]]
ranks = blender.rank(inputs, candidates_texts, return_scores=False, batch_size=1)

"""
ranks -->
array([[2, 1, 3], # it means "The meeting is at 2 PM." ranks 1st, "It's at 10 AM." ranks 2nd, and "I'll check and let you know." ranks 3rd.
       [1, 2, 3]], # it means "Try 'The Great Gatsby'." ranks 1st, "How about '1984'?" ranks 2nd, and "I’m not sure, maybe check online reviews." ranks 3rd.
       dtype=int32)
"""



Ranking candidates: 100%|██████████| 2/2 [00:01<00:00,  1.92it/s]


'\nranks -->\narray([[2, 1, 3], # it means "The meeting is at 2 PM." ranks 1st, "It\'s at 10 AM." ranks 2nd, and "I\'ll check and let you know." ranks 3rd.\n       [1, 2, 3]], # it means "Try \'The Great Gatsby\'." ranks 1st, "How about \'1984\'?" ranks 2nd, and "I’m not sure, maybe check online reviews." ranks 3rd.\n       dtype=int32)\n'

In [ ]:
ranks

array([[2, 1, 3],
       [1, 2, 3]], dtype=int32)

### Preparing the dataset for use in PairRM

We need two lists, one a list of instructions and the other is a list of lists, it contains the set of responses for each instruction.

In [ ]:
import json

with open('generated_responses.json', 'r') as f:
  generated_responses = json.load(f)

with open('50_sampled_conversations.json', 'r') as f:
  sampled_instructions = json.load(f)

In [ ]:
sampled_instructions[0]

'Can I spend the night alone in a tent in a forest outside Stockholm in -20°C without risking my life?\n\nThe backstory\nFrom the end of January, I\'m starting my studies in a suburb of Stockholm. I\'ve decided to, if it turns out plausible, not rent an apartment, but live in a tent. (This is not out of frugality, but out of a will to try something new.)\nI do have friends who I could visit once a week or so to prepare food and wash my clothes, so I think I can solve the practical problems, or at least those that I\'ve come to think of. I\'d camp in one of the forests, maybe 1 km from "civilisation". I\'d have access to showers etc at university every day.\nHowever: I don\'t want to freeze to death in my sleep! That\'s very important to me. I\'ve read that the nights can get as cold as -20°C (-4°F). With the proper preparations, would this be a plausible way of living, at least for a month or so?\nI do have camping experience, and have been hiking for three weeks, but only in summer.'

In [ ]:
generated_responses["1"]

["Living in a tent in a forest outside Stockholm during very cold temperatures, such as -20°C (-4°F), can be challenging and potentially dangerous if not properly prepared. Here are some considerations and tips based on your situation:\n\n1. Shelter: Ensure your tent is well-insulated and rated for cold weather. Look for a four-season rating, which indicates it can handle extreme temperatures. Using an insulated sleeping pad under your mattress and a warm sleeping bag with a temperature rating below -20°C can help you stay warm during the coldest nights.\n\n2. Cooking and Heating: Use a proper camping stove to cook and heat up water for hygiene purposes and making hot drinks. Keep in mind that fuel may not flow well in the extreme cold, so make sure to buy enough fuel and take proper precautions to avoid fire risks or running out of fuel.\n\n3. Fire: Having a fire as a secondary heat source can help you stay warm during extremely cold nights. Use a fire pit or a portable stove to prepa

In [ ]:
reIndexedResponses = []

# We need to strip the '</s>' from the responses
for index in generated_responses:

    responses = generated_responses[index]

    strippedResponses = []
    for response in responses:
        strippedResponses.append(response.replace('</s>', ''))

    reIndexedResponses.append(strippedResponses)

In [ ]:
len(reIndexedResponses)

50

In [ ]:
reIndexedResponses[0]

["Living in a tent in a forest outside Stockholm during very cold temperatures, such as -20°C (-4°F), can be challenging and potentially dangerous if not properly prepared. Here are some considerations and tips based on your situation:\n\n1. Shelter: Ensure your tent is well-insulated and rated for cold weather. Look for a four-season rating, which indicates it can handle extreme temperatures. Using an insulated sleeping pad under your mattress and a warm sleeping bag with a temperature rating below -20°C can help you stay warm during the coldest nights.\n\n2. Cooking and Heating: Use a proper camping stove to cook and heat up water for hygiene purposes and making hot drinks. Keep in mind that fuel may not flow well in the extreme cold, so make sure to buy enough fuel and take proper precautions to avoid fire risks or running out of fuel.\n\n3. Fire: Having a fire as a secondary heat source can help you stay warm during extremely cold nights. Use a fire pit or a portable stove to prepa

In [ ]:
print(len(results))
# print(results[0])

50


In [ ]:
results[1]

["Living in a tent in a forest outside Stockholm during very cold temperatures, such as -20°C (-4°F), can be challenging and potentially dangerous if not properly prepared. Here are some considerations and tips based on your situation:\n\n1. Shelter: Ensure your tent is well-insulated and rated for cold weather. Look for a four-season rating, which indicates it can handle extreme temperatures. Using an insulated sleeping pad under your mattress and a warm sleeping bag with a temperature rating below -20°C can help you stay warm during the coldest nights.\n\n2. Cooking and Heating: Use a proper camping stove to cook and heat up water for hygiene purposes and making hot drinks. Keep in mind that fuel may not flow well in the extreme cold, so make sure to buy enough fuel and take proper precautions to avoid fire risks or running out of fuel.\n\n3. Fire: Having a fire as a secondary heat source can help you stay warm during extremely cold nights. Use a fire pit or a portable stove to prepa

In [ ]:
ranks_generated_responses = blender.rank(sampled_instructions, reIndexedResponses, return_scores=False, batch_size=1)


Ranking candidates: 100%|██████████| 50/50 [02:23<00:00,  2.87s/it]


**Here we see that, the PairRM's rank call has ranked the given options and given each a score. Now let's convert this to a format that can be saved as a dataset for the DPO task.**

We will be using this dataset as reference for the format and structure. [snorkelai/Snorkel-Mistral-PairRM-DPO-Dataset](https://huggingface.co/datasets/snorkelai/Snorkel-Mistral-PairRM-DPO-Dataset).

Then we will push this to Hugging Face

In [ ]:
ranks_generated_responses[0]

array([2, 4, 1, 5, 3], dtype=int32)

In [ ]:
response_rank_json = {}

for index in range(len(ranks_generated_responses)):
    response_rank_json[index+1] = ranks_generated_responses[index].tolist()

In [ ]:
response_rank_json

{1: [2, 4, 1, 5, 3],
 2: [1, 2, 3, 5, 4],
 3: [2, 3, 4, 5, 1],
 4: [5, 4, 3, 1, 2],
 5: [5, 2, 1, 3, 4],
 6: [3, 2, 1, 5, 4],
 7: [5, 4, 3, 1, 2],
 8: [2, 1, 3, 5, 4],
 9: [2, 3, 5, 4, 1],
 10: [3, 5, 4, 1, 2],
 11: [4, 1, 3, 2, 5],
 12: [4, 1, 2, 3, 5],
 13: [4, 1, 5, 3, 2],
 14: [2, 4, 3, 1, 5],
 15: [1, 2, 3, 4, 5],
 16: [5, 4, 2, 3, 1],
 17: [4, 1, 5, 2, 3],
 18: [3, 2, 4, 1, 5],
 19: [2, 1, 3, 4, 5],
 20: [2, 4, 1, 5, 3],
 21: [5, 1, 3, 4, 2],
 22: [5, 3, 2, 4, 1],
 23: [3, 5, 4, 2, 1],
 24: [5, 4, 3, 2, 1],
 25: [3, 4, 1, 5, 2],
 26: [5, 4, 3, 2, 1],
 27: [4, 3, 5, 2, 1],
 28: [1, 3, 4, 5, 2],
 29: [1, 3, 4, 5, 2],
 30: [3, 1, 2, 5, 4],
 31: [2, 5, 1, 4, 3],
 32: [5, 4, 3, 1, 2],
 33: [1, 4, 5, 2, 3],
 34: [1, 2, 3, 5, 4],
 35: [2, 3, 1, 4, 5],
 36: [5, 1, 4, 2, 3],
 37: [3, 2, 4, 1, 5],
 38: [4, 2, 1, 3, 5],
 39: [4, 2, 5, 1, 3],
 40: [3, 4, 1, 2, 5],
 41: [4, 3, 2, 1, 5],
 42: [4, 2, 1, 3, 5],
 43: [3, 5, 1, 2, 4],
 44: [1, 2, 5, 3, 4],
 45: [1, 3, 5, 4, 2],
 46: [2, 3, 5, 4, 1

In [ ]:
import json
with open('ranked_responses.json', 'w') as f:
    json.dump(response_rank_json, f, indent=4)

**Creating the Preference Dataset**

To construct the preference dataset, follow these steps:

- **`prompt_id`**: Unique identifier (hash) – This needs to be generated.
- **`prompt`**: Text prompt – This is already available.
- **`chosen`**: List of chosen responses – This needs to be generated.
- **`rejected`**: List of rejected responses – This needs to be generated.
- **`all_generated_responses`**: Sequence of all generated responses – This is already available as a list.
- **`all_rm_scores`**: Sequence of ranking scores – Append values directly from results.


In [ ]:
import hashlib

def create_md5_hash(input_string):
    # Create a new MD5 hash object
    hasher = hashlib.md5()
    # Update the hash object with the bytes of the string
    hasher.update(input_string.encode('utf-8'))
    # Obtain the digest, which is the hash value as a hexadecimal string
    return hasher.hexdigest()

In [ ]:
preference_dataset_list = []

for index in range(len(sampled_instructions)):
    prompt = sampled_instructions[index]
    prompt_id = create_md5_hash(prompt)
    all_generated_resopnses = reIndexedResponses[index]
    all_rm_scores = response_rank_json[index+1] #.tolist()

    max_value = max(all_rm_scores)
    max_index = all_rm_scores.index(max_value)

    min_score = min(all_rm_scores)
    min_index = all_rm_scores.index(min_score)


    chosen = [{"content": prompt, "role": "user"}, {"content": all_generated_resopnses[max_index], "role": "assistant"}]
    rejected = [{"content": prompt, "role": "user"}, {"content": all_generated_resopnses[min_index], "role": "assistant"}]

    json_entry = dict()
    json_entry['prompt_id'] = prompt_id
    json_entry['prompt'] = prompt
    json_entry['chosen'] = chosen
    json_entry['rejected'] = rejected
    json_entry['all_generated_resopnses'] = all_generated_resopnses
    json_entry['all_rm_scores'] = all_rm_scores

    preference_dataset_list.append(json_entry)

In [ ]:
preference_dataset_list[0]

{'prompt_id': '57563f0b803851479c4e8a632b6016fc',
 'prompt': 'Can I spend the night alone in a tent in a forest outside Stockholm in -20°C without risking my life?\n\nThe backstory\nFrom the end of January, I\'m starting my studies in a suburb of Stockholm. I\'ve decided to, if it turns out plausible, not rent an apartment, but live in a tent. (This is not out of frugality, but out of a will to try something new.)\nI do have friends who I could visit once a week or so to prepare food and wash my clothes, so I think I can solve the practical problems, or at least those that I\'ve come to think of. I\'d camp in one of the forests, maybe 1 km from "civilisation". I\'d have access to showers etc at university every day.\nHowever: I don\'t want to freeze to death in my sleep! That\'s very important to me. I\'ve read that the nights can get as cold as -20°C (-4°F). With the proper preparations, would this be a plausible way of living, at least for a month or so?\nI do have camping experience

In [ ]:
# Convert list of dictionaries to a format suitable for creating a Dataset
data_dict = {key: [dic[key] for dic in preference_dataset_list] for key in preference_dataset_list[0]}

In [ ]:
from datasets import Dataset

# Create a Hugging Face dataset
dataset = Dataset.from_dict(data_dict)

# Output the dataset to check it
print(dataset)

Dataset({
    features: ['prompt_id', 'prompt', 'chosen', 'rejected', 'all_generated_resopnses', 'all_rm_scores'],
    num_rows: 50
})


In [ ]:
dataset.push_to_hub("Mistral-7B-Instruct-v2.0-Lima-PairRM-DPO-Dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jkkn/Mistral-7B-Instruct-v2.0-Lima-PairRM-DPO-Dataset/commit/daee4b3713854ddc956ee50e711c74603299c046', commit_message='Upload dataset', commit_description='', oid='daee4b3713854ddc956ee50e711c74603299c046', pr_url=None, pr_revision=None, pr_num=None)